In [1]:
from sqlalchemy import create_engine
import pandas as pd
from getpass import getpass  # To get the password without showing the input

In [2]:
password = getpass("PASSWORD: ")

In [3]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/bank'
engine = create_engine(connection_string)

1. make sure it reads the data from the tables

In [4]:
rental_may = pd.read_sql_query('SELECT * FROM sakila.rental_may', engine)
rental_may.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [5]:
rental_june = pd.read_sql_query('SELECT * FROM sakila.rental_june', engine)
rental_june.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


2. count number of rentals per customer_id

In [6]:
query_may = "SELECT customer_id, COUNT(rental_id) AS rented_may FROM sakila.rental_may GROUP BY customer_id"
rental_may = pd.read_sql_query(query_may, engine)
rental_may.head()

,customer_id,rented_may
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3


In [7]:
query_june = "SELECT customer_id, COUNT(rental_id) AS rented_june FROM sakila.rental_june GROUP BY customer_id"
rental_june = pd.read_sql_query(query_june, engine)
rental_june.head()

,customer_id,rented_june
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5


3. merge data frames and create new column to measure the difference

In [8]:
rental_merge = rental_may.merge(rental_june, on="customer_id", how="inner")
rental_merge.head()

,customer_id,rented_may,rented_june
0,1,2,7
1,2,1,1
2,3,2,4
3,5,3,5
4,6,3,4


In [11]:
#cannot use for loops - must use a conditional formula as below
rental_merge["compar_may_june"]="Decrease in rental"
rental_merge["compar_may_june"][rental_merge['rented_may']<rental_merge['rented_june']] ="Increase in rental"
rental_merge["compar_may_june"][rental_merge['rented_may']==rental_merge['rented_june']] ="Equal"

C:\Users\jordi\AppData\Local\Temp\ipykernel_4856\1292738306.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rental_merge["compar_may_june"][rental_merge['rented_may']<rental_merge['rented_june']] ="Increase in rental"
C:\Users\jordi\AppData\Local\Temp\ipykernel_4856\1292738306.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rental_merge["compar_may_june"][rental_merge['rented_may']==rental_merge['rented_june']] ="Equal"


In [12]:
rental_merge.sample(15)

,customer_id,rented_may,rented_june,compar_may_june
133,157,2,1,Decrease in rental
251,298,1,4,Increase in rental
405,474,1,2,Increase in rental
469,546,2,4,Increase in rental
336,396,1,3,Increase in rental
182,217,2,3,Increase in rental
510,597,2,3,Increase in rental
48,58,3,3,Equal
447,522,1,4,Increase in rental
137,162,3,5,Increase in rental
